In [1]:
!pip install transformers datasets torch scikit-learn evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.5 MB/s eta 0:00:00


In [24]:
# Importing dataset

from datasets import load_dataset

dataset_name = "Keyurjotaniya007/datasets-emotion-2.0" # Balanced classes
dataset = load_dataset(dataset_name)

In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = "distilbert-base-uncased"
tokenizer =  AutoTokenizer.from_pretrained(model)

id2label = {0: "anger", 1: "fear", 2: "joy", 3: "love", 4: "sadness", 5: "surprise"}
label2id = {"anger": 0, "fear": 1, "joy": 2, "love": 3, "sadness": 4, "surprise": 5}

model = AutoModelForSequenceClassification.from_pretrained(model, num_labels=6, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
# Freeze all base model paramaters
#for name, param in model.base_model.named_parameters():
   # param.requires_grad = False

# Unfreeze pooling layers
#for name, param in model.base_model.named_parameters():
 #   if "pooler" in name:
  #      param.requires_grad = True

#for name, param in model.base_model.named_parameters():
 #   if any(layer_id in name for layer_id in ["layer.0", "layer.1", "layer.2"]):
  #      param.requires_grad = False

for name, param in model.base_model.named_parameters():
    if "embeddings" in name:
        param.requires_grad = False

In [55]:
# Preprocessing Data
def preprocess_data(text):
  return tokenizer(text=text["text"], truncation=True)

tokenized_data = dataset.map(preprocess_data, batched=True)

In [56]:
# Data Collator

from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [57]:
import evaluate
import numpy as np

accuracy_metric = evaluate.load("accuracy")
auc_metric = evaluate.load("roc_auc")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    probs = np.exp(logits) / np.exp(logits).sum(axis=1, keepdims=True)
    preds = np.argmax(probs, axis=1)

    acc = accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]

    f1 = f1_metric.compute(predictions=preds, references=labels, average="weighted")["f1"]

    try:
        auc = auc_metric.compute(prediction_scores=probs, references=labels, multi_class="ovr", average="macro")["roc_auc"]
    except Exception:
        auc = np.nan

    return {"accuracy": round(acc, 3), "f1": round(f1, 3), "auc": round(auc, 3) if auc is not np.nan else auc}


In [58]:
from transformers import TrainingArguments, Trainer

# Hyperparameters
lr = 2e-5
epochs = 6
batch_size = 16

training_args = TrainingArguments(
    output_dir="emotion_model",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.05,
    warmup_ratio=0.1,
    fp16=True,
    lr_scheduler_type="linear",
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True
)

In [59]:
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

/tmp/ipython-input-3223423925.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Auc
1,0.014300,0.751071,0.919000,0.919000,nan
2,0.024800,0.531304,0.925000,0.925000,nan
3,0.022500,0.540359,0.921000,0.921000,nan
4,0.013100,0.568992,0.920000,0.919000,nan


TrainOutput(global_step=16088, training_loss=0.018682576126864987, metrics={'train_runtime': 970.6666, 'train_samples_per_second': 397.731, 'train_steps_per_second': 24.861, 'total_flos': 1.5524549911920672e+16, 'train_loss': 0.018682576126864987, 'epoch': 4.0})

In [60]:
predictions = trainer.predict(tokenized_data["validation"])

logits = predictions.predictions
labels = predictions.label_ids

metrics = compute_metrics((logits, labels))
print(metrics)

{'accuracy': 0.928, 'f1': 0.928, 'auc': nan}


In [61]:
trainer.state.best_model_checkpoint

best_model_path = trainer.state.best_model_checkpoint

trainer.save_model("emotion_model_best")
tokenizer.save_pretrained("emotion_model_best")

('emotion_model_best/tokenizer_config.json',
 'emotion_model_best/special_tokens_map.json',
 'emotion_model_best/vocab.txt',
 'emotion_model_best/added_tokens.json',
 'emotion_model_best/tokenizer.json')

In [62]:
from google.colab import files
import shutil

shutil.make_archive("emotion_model_best", "zip", "emotion_model_best")
files.download("emotion_model_best.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

{'accuracy': 0.924, 'f1': 0.924, 'auc': nan}

{'accuracy': 0.921, 'f1': 0.921, 'auc': nan}

{'accuracy': 0.928, 'f1': 0.928, 'auc': nan}